Importing Necessary Libraries

In [26]:
import random
import re
from transformers import pipeline


Defining Models

In [2]:
model = pipeline(
    task="zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli"
)
ner = pipeline(
    task="ner",
    model="dslim/bert-base-NER",
    aggregation_strategy="simple"
)

Device set to use cpu


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


ommon patterns to identify intents

In [3]:
label =["social conversation","factual information lookup"]
chat_content = ["hi","hello","thanks","thank you"]
search_content = ["who", "what", "when", "where", "how does","tell me about"]
SOCIAL_ACT_PATTERNS = {
  "GREETING": ["hi", "hello", "hey"],
  "GRATITUDE": ["thank", "thanks"],
  "PERMISSION_REQUEST": ["can i ask", "may i ask"],
  "WELLBEING_QUERY": ["how are you", "how do you feel"],
}
SOCIAL_ACT_RESPONSES = {
  "GREETING": [
      "Hey 🙂 What can I help you with?",
      "Hi there! How can I assist?"
  ],
  "GRATITUDE": [
      "You're welcome!",
      "Glad I could help 🙂"
  ],
  "PERMISSION_REQUEST": [
      "Of course. Go ahead.",
      "Sure, what do you want to ask?"
  ],
  "WELLBEING_QUERY": [
      "I'm doing well — how can I help you today?",
      "I'm just here to help you 🙂"
  ]
}
INFO_VERBS = ["explain", "tell", "describe", "define"]

In [28]:

def looks_like_info_request(text):
    return any(v in text for v in INFO_VERBS)

def route_intent(text):
  result=model(text,label)

  top_labels,top_scores=result["labels"],result["scores"]
  if(any(x in text for x in chat_content)):
    final_decision = "social conversation"
  elif(any(x in text for x in search_content)):
    final_decision = "factual information lookup"
  else:
    if(top_scores[0]<=.55):
      final_decision = "factual information lookup"
    else:
      final_decision = top_labels[0]

  return final_decision

def handle_social_conversation(text):
  for key,value in SOCIAL_ACT_PATTERNS.items():
    if any(x in text for x in value):
      return key
  return None

def handle_factual_information_lookup(text):
  print("Let me look that up for you.")
  return "(Search results will appear here)"

def extract_entities(text):
  entities = ner(text)
  values={}

  for entity in entities:
    label = entity["entity_group"]
    word = entity["word"]

    values.setdefault(label, []).append(word)
  return values

def debug_decision(raw_text, intent, entities, response=None):
    return {
        "input": raw_text,
        "intent": intent,
        "response": response,
        "entities": entities
    }
def clean_text(text):

  cleaned_text = re.sub(r'\[.*?\]', '', text)
  cleaned_text = cleaned_text.replace("\r\n","\n").replace("\r","\n")
  cleaned_text = cleaned_text.strip()

  return cleaned_text

def chunk_document(text):

  raw_chunk = re.split(r'\n{2,}',text)
  refined_chunk=[]

  for index,chunk in enumerate(raw_chunk):
    chunk = chunk.strip()

    if len(chunk) < 50:
      continue
    refined_obj={
      "chunk_id":f"One_Piece_overview_{index}",
      "chunk":chunk,
      "chunk_length":len(chunk),
      "num_sentences":len(re.findall(r'[.!?]+', chunk)),
      "position":index,
      "entities":extract_entities(chunk)
    }
    
    refined_chunk.append(refined_obj)

  return refined_chunk



In [ ]:


while True:
    raw_text = input("what do you have in mind today? ")
    normalized_text = raw_text.lower()
    if looks_like_info_request(normalized_text):
        intent = "factual information lookup"
    else:
        intent = route_intent(normalized_text)

    print(intent)

    if intent == "social conversation":
        social_act = handle_social_conversation(normalized_text)
        if social_act:
            response = random.choice(SOCIAL_ACT_RESPONSES[social_act])
        else:
            response = random.choice(SOCIAL_ACT_RESPONSES["GREETING"])
    else:
        response = handle_factual_information_lookup(raw_text)
    entities = extract_entities(raw_text)
    debug = debug_decision(raw_text, intent, entities, response)
    print(debug)
    print("press 1 to exit")
    if input() == "1":
        break



In [ ]:
RAW_DOC = """
Artificial Intelligence is a field of computer science focused on building systems that can perform tasks requiring human-like intelligence. 
These tasks include reasoning, learning, perception, and language understanding.

Machine Learning is a subfield of AI that enables systems to learn patterns from data instead of being explicitly programmed. 
It is widely used in recommendation systems, fraud detection, and speech recognition.

Natural Language Processing allows machines to understand and generate human language. 
Applications include chatbots, search engines, and automated customer support.
"""

cleaned_text = clean_text(RAW_DOC)
chunks=chunk_document(cleaned_text)
document = {
    "doc_id": "Document_overview",
    "source": "wikipedia",
    "raw_text": RAW_DOC,
    "clean_text": cleaned_text,
    "chunks": chunks,
    "No. of chunks""":len(chunks)

    }
print(document)

{'doc_id': 'One-Piece_overview', 'source': 'wikipedia', 'raw_text': '\nArtificial Intelligence is a field of computer science focused on building systems that can perform tasks requiring human-like intelligence. \nThese tasks include reasoning, learning, perception, and language understanding.\n\nMachine Learning is a subfield of AI that enables systems to learn patterns from data instead of being explicitly programmed. \nIt is widely used in recommendation systems, fraud detection, and speech recognition.\n\nNatural Language Processing allows machines to understand and generate human language. \nApplications include chatbots, search engines, and automated customer support.\n', 'clean_text': 'Artificial Intelligence is a field of computer science focused on building systems that can perform tasks requiring human-like intelligence. \nThese tasks include reasoning, learning, perception, and language understanding.\n\nMachine Learning is a subfield of AI that enables systems to learn patt